# Inferencia en LDA
El problema de inferencia a resolver para usar LDA es el cálculo de la distribución a posteriori:
$$p(\theta, {\bf z} \mid {\bf w},\alpha,\beta) = \frac{p(\theta, {\bf z} , {\bf w}\mid \alpha,\beta)}{p({\bf w}\mid \alpha,\beta)}$$
dónde 
$$p(\theta, {\bf z}, {\bf w}\mid \alpha,\beta)= p(\theta \mid \alpha) \prod_{n=1}^N p(z_n\mid \theta) p(w_n\mid z_n,\beta)$$
siendo $N$ el número de palabras en el documento,
$${\bf z} = (z_1,...,z_N),\qquad {\bf w} = (w_1,...,w_N)$$

$$z_n = (0,0,...,1,...,0) \in \{0,1\}^K$$ con 
$z_n^i =  1$ si la n-ésima palabra es asignada al tópico i, 
0 si no.
$$\,$$
$$z_n \sim Multinomial(\theta)$$ entonces
$$p(z_n \mid \theta) = \frac{1!}{0!...1!...0!}\prod_{i=1}^K \theta^{z_n^i
}= \theta_i \qquad cuando \qquad z_n^i = 1$$


De manera equivalente se define
$$w_n = (0,0,...,1,...,0) \in \{0,1\}^V$$ con 
$w_n^j =  1$ si la n-ésima palabra corresponde a la j-ésima palabra del vocabulario, 
0 si no.
$$\,$$
Por otra parte, se define 
$$\beta_{ij} = p(w_n^j = 1 \mid z_n^i =1),\qquad i = 1,...K, j= 1,...,V$$
Así,

$$p(\theta, {\bf z}, {\bf w}\mid \alpha,\beta)= p(\theta \mid \alpha) \prod_{n=1}^N \left(\prod_{i=1}^K \left[\theta_i \prod_{j=1}^V \beta_{ij}^{w_n^j}\right]^{z_n^i}\right)$$
De manera que al integrar sobre todos los valores posibles de $\theta$ y sumar sobre todos los valores posibles de ${\bf z}$ se obtiene:
$$p({\bf w}\mid \alpha,\beta)= \int_{\Theta} \sum_{\bf z} p(\theta \mid \alpha) \prod_{n=1}^N \left(\prod_{i=1}^K \left[\theta_i \prod_{j=1}^V \beta_{ij}^{w_n^j}\right]^{z_n^i}\right)d\theta$$
Los valores posibles de ${\bf z} = (z_1,...,z_N)$ dependen de los $z_i$ que independientes uno de otros toman $K$ valores posibles. Entonces:
$$p({\bf w}\mid \alpha,\beta)= \int_{\Theta}  p(\theta \mid \alpha) \prod_{n=1}^N \left(\sum_{i=1}^K \left[\prod_{j=1}^V (\theta_i\beta_{ij})^{w_n^j}\right]\right )d\theta$$

Como $\theta \sim Dirichlet(\alpha)$ entonces
$$p({\bf w}\mid \alpha,\beta)= \frac{\Gamma(\sum_{i=1}^K \alpha_i)}{\prod_{i=1}^K \Gamma(\alpha_i)} \int_{\Theta} \left(\prod_{i=1}^K \theta^{\alpha_i -1}\right) \left( \prod_{n=1}^N \left(\sum_{i=1}^K \prod_{j=1}^V (\theta_i\beta_{ij})^{w_n^j}\right)\right )d\theta$$
que es intratable debido al acoplamiento entre $\beta$ y $\theta$ en la suma de tópicos latentes.




# Inferencia Variacional
(Ver <a href= "https://drive.google.com/open?id=1BobImO3192hifZPLXowd14gryVAUzBPW"> Latent Dirichlet Allocation (LDA) </a>)
$$\,$$
La idea básica de la inferencia variacional es definir una familia de funciones parametrizadas que acoten inferiormente a la distribución a posteriori, y entonces la inferencia se transforma en un problema de maximización de esa familia. 

Para ello se propone una familia de distribuciones a priori en que se encuentran desacopladas $\theta$ y ${\bf z}$, es decir
$$ \theta \sim Dirichlet(\gamma)$$
y
$$ {\bf z} \sim Multinomial(\phi)$$
Así, 
$$q(\theta,{\bf z} \mid \gamma,\phi) = q(\theta \mid \gamma)\prod_{n=1}^N q(z_n \mid \phi_n)$$
permite definir una familia de funciones que acota inferiormente el logaritmo de la distribución predictiva o evidencia.
En efecto
$$log p({\bf w} \mid \alpha,\beta) = log\int_\Theta \sum_{\bf z} p(\theta, {\bf z}, {\bf w}\mid \alpha,\beta)d\theta$$
$$ = log\int_\Theta \sum_{\bf z} \frac{p(\theta, {\bf z}, {\bf w}\mid \alpha,\beta)}{q(\theta,{\bf z})} q(\theta,{\bf z})d\theta$$
$$ = log \mathop{\mathbb{E}_q}\left[\frac{p(\cdot, \cdot, {\bf w}\mid \alpha,\beta)}{q(\cdot,\cdot)}\right] $$
por la desigualdad de Jensen para el caso de funciones cóncavas (caso del logaritmo):
$$\geq \mathop{\mathbb{E}_q} log\left[\frac{p(\cdot, \cdot, {\bf w}\mid \alpha,\beta)}{q(\cdot,\cdot)}\right] $$
$$ = \mathop{\mathbb{E}_q}\left[log p(\cdot, \cdot, {\bf w}\mid \alpha,\beta)\right] - \mathop{\mathbb{E}_q}\left[log q(\cdot,\cdot)\right] $$

Antes de proseguir, vamos a calcular la divergencia de Kullback-Leibler entre la distribución a posteriori y q:
$$D_{KL}(q \mid p) = \mathop{\mathbb{E}_q}\left[log \frac{q(\cdot,\cdot)}{p(\cdot, \cdot \mid {\bf w} ,\alpha,\beta)}\right] $$
$$\,$$
$$= \mathop{\mathbb{E}_q}\left[log q(\cdot,\cdot)\right] -  \mathop{\mathbb{E}_q}\left[log p(\cdot, \cdot\mid {\bf w}, \alpha,\beta)\right] $$
$$\,$$
$$= \mathop{\mathbb{E}_q}\left[log q(\cdot,\cdot)\right] -  \mathop{\mathbb{E}_q}\left[log \frac{p(\cdot, \cdot , {\bf w}\mid \alpha,\beta)}{p({\bf w}\mid \alpha,\beta)}\right] $$
$$\,$$
$$= \mathop{\mathbb{E}_q}\left[log q(\cdot,\cdot)\right] -  \mathop{\mathbb{E}_q}\left[log p(\cdot, \cdot , {\bf w}\mid \alpha,\beta)\right] + \mathop{\mathbb{E}_q}\left[log p({\bf w}\mid \alpha,\beta)\right] $$
$$\,$$
$$= \mathop{\mathbb{E}_q}\left[log q(\cdot,\cdot)\right] -  \mathop{\mathbb{E}_q}\left[log p(\cdot, \cdot , {\bf w}\mid \alpha,\beta)\right] + log p({\bf w}\mid \alpha,\beta) $$
$$\,$$
Es decir,
si denominamos
$$L(\gamma,\phi,\alpha,\beta) = \mathop{\mathbb{E}_q}\left[log p(\cdot, \cdot, {\bf w}\mid \alpha,\beta)\right] - \mathop{\mathbb{E}_q}\left[log q(\cdot,\cdot\mid \gamma,\phi)\right]$$
tenemos que
$$log p({\bf w}\mid \alpha,\beta) = L(\gamma,\phi,\alpha,\beta) + D_{KL}(q \mid p) $$
Entonces, minimizar $D_{KL}(q \mid p)$
es equivalente a maximizar L.
El problema de inferencia se transforma en el problema de maximización:
$$\mathop{max}_{\gamma,\phi} L(\gamma,\phi,\alpha,\beta)$$
Ver en los anexos A3.1 y A3.2 del artículo en referencia los detalles de la resolución de este problema de maximización, asi como en A4 las explicaciones de como se utiliza la estrategia EM para encontrar los estimadores de Bayes empíricos  para $\alpha$ y $\beta$.
$$\,$$
Posteriormente, en Blei et al(2011) se propone un nuevo algoritmo de inferencia variacional online para el modelo LDA.



# Inferencia con Gibbs Sampling
(Ver <a href= "https://drive.google.com/file/d/1oSRRvG_W0UaH0l4o9f5ClEyWuGY1j-s0/view?usp=sharing"> Gibbs sampling para Latent Dirichlet Allocation (LDA) </a>)
$$\,$$
Para la formulación de la inferencia utilizando Gibbs sampling, es necesario considerar una distribución a priori Dirichlet sobre el parámetro $\beta$ que representa la distribución de tópicos en cada documento. Tenemos entonces:
$$w_n \mid z_n, \beta^{(z_n)} \sim Multinomial(\beta^{(z_n)})$$
$$\beta \sim Dirichlet(\eta)$$
$$ z_n \mid \theta \sim Multinomial(\theta)$$
$$ \theta \sim Dirichlet(\alpha)$$
Entonces
$$p({\bf w},{\bf z} \mid \theta,\beta) = p({\bf w}  \mid {\bf z}, \theta,\beta)p({\bf z} \mid \theta)$$
de dónde
$$\int_{\Theta} \int_{\Gamma} p({\bf w},{\bf z} \mid \theta,\beta)p(\beta)d\beta p(\theta) d\theta = \int_{\Gamma} p({\bf w}  \mid {\bf z}, \beta)p(\beta)d\beta \int_{\Theta} p({\bf z} \mid \theta)p(\theta) d\theta$$

Si consideramos
$$\eta = (\eta_1,...,\eta_V) = (\eta,...,\eta)$$
y
$$\alpha = (\alpha_1,...,\alpha_K) = (\alpha,...,\alpha)$$
entonces:

$$p({\bf z}) = \int_{\Theta} p({\bf z} \mid \theta)p(\theta) d\theta$$
$$= \left(\frac{\Gamma(K\alpha)}{(\Gamma(\alpha))^K}\right)^D \prod_{d=1}^D \prod_{i=1}^K \frac{\Gamma(n_i^{(d)} + \alpha)}{\Gamma(n_{\cdot}^{(d)} + K\alpha)}$$
con $n_i^{(d)}$ es el número de veces que una palabra del documento fue asignada al tópico $i$.
$$\,$$
$$p({\bf w} \mid {\bf z}) = \int_{\Gamma} p({\bf w}  \mid {\bf z}, \beta)p(\beta)d\beta$$
$$= \left(\frac{\Gamma(V\eta)}{(\Gamma(\eta))^V}\right)^K \prod_{i=1}^K
\prod_{w \in W} \frac{\Gamma(n_i^{(w)} + \eta)}{\Gamma(n_i^{(\cdot)} + K\eta)}$$
dónde $n_i^{(w)}$ es el número de veces que una palabra $w$ es asignada al tópico $i$ en el vector de asignaciones ${\bf z}$.
Con estas especificaciones, el cálculo de la distribución a posteriori:

$$p({\bf z} \mid {\bf w}) =\frac{p({\bf w} \mid {\bf z})p({\bf z})}{\sum_{\bf z} p({\bf w} \mid {\bf z})p({\bf z})}$$

es intratable (la suma sobre ${\bf z}$ es del orden de $V^N$).

Entonces se considera el uso de Gibbs sampling aplicando el algoritmo a 
$$p(z_j \mid z_{-j} , {\bf w})$$
donde $z_j$ es el tópico asignado a la j-ésima palabra y $z_{-j}$ son los tópicos asignados a las demás palabras del documento. 

A grandes rasgos, Gibbs sampling consiste en generar muestras de estas distribuciones condicionales, que tienen la particularidad de converger a la distribución a posteriori de ${\bf z}$.


Notar que dichas distribuciones condicionales tienen expresión analítica debido a las propiedades de las distribuciones naturales conjugadas: 
$$p(z_j \mid z_{-j} , {\bf w}) \propto \frac{n_{-j,i}^{(w_j)} + \eta}{n_{-j,i}^{(\cdot)} + V\eta} \frac{n_{-j,i}^{(d_j)} + \alpha}{n_{-j,\cdot}^{(d_j)} + K\alpha}$$
donde
$n_{-j,i}^{(w_j)}$ es el número de veces que $w_j$ (aparte de la j-ésima palabra) aparece en el corpus asignada al tópico i.
$$\,$$
$n_{-j,i}^{(d_j)}$ es el número de palabras en $d_j$ (excepto la j-ésima palabra)  asignada al tópico i.




